In [ ]:
import time
import asyncio
from transformers import AutoTokenizer
from vllm import SamplingParams, AsyncEngineArgs, AsyncLLMEngine
from typing import List, Dict
import os

In [ ]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#%%
MODEL_PATH = os.environ.get('MODEL_PATH', '/data/zhenyu/LLM_Model/GLM_4/GLM4_Chat/')
max_length = 512
top_p = 1
temperature = 0

In [ ]:
def load_model_and_tokenizer(model_dir: str):
    engine_args = AsyncEngineArgs(
        model=model_dir,
        tokenizer=model_dir,
        tensor_parallel_size=1,
        dtype="bfloat16",
        trust_remote_code=True,
        gpu_memory_utilization=1,
        enforce_eager=True,
        worker_use_ray=True,
        engine_use_ray=False,
        disable_log_requests=True
        # 如果遇见 OOM 现象，建议开启下述参数
        # enable_chunked_prefill=True,
        # max_num_batched_tokens=8192
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir,
        trust_remote_code=True,
        encode_special_tokens=True
    )
    engine = AsyncLLMEngine.from_engine_args(engine_args)
    return engine, tokenizer

async def vllm_gen(messages: List[Dict[str, str]], top_p: float, temperature: float, max_dec_len: int):
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )
    params_dict = {
        "n": 1,
        "best_of": 1,
        "presence_penalty": 1.0,
        "frequency_penalty": 0.0,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": -1,
        "use_beam_search": False,
        "length_penalty": 1,
        "early_stopping": False,
        "stop_token_ids": [151329, 151336, 151338],
        "ignore_eos": False,
        "max_tokens": max_dec_len,
        "logprobs": None,
        "prompt_logprobs": None,
        "skip_special_tokens": True,
    }
    sampling_params = SamplingParams(**params_dict)
    async for output in engine.generate(inputs=inputs, sampling_params=sampling_params, request_id=f"{time.time()}"):
        yield output.outputs[0].text

def perdict(messages):
    current_length = 0
    output = ""
    for output in vllm_gen(messages, top_p, temperature, max_length):
        print(output[current_length:], end="", flush=True)
        current_length = len(output)
    return output

engine, tokenizer = load_model_and_tokenizer(MODEL_PATH)

In [ ]:
system_prompt_1 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
CAC > 9, CTAI 结节风险 > 85%：分析解读参考 “ CAC、CTAI均在高风险区间，应选择积极治疗，特别是结节大小≥8mm，手术干预指征更强。如果结节＜8mm，或患者拒绝手术治疗时，可选择密切观察（每3个月复查胸部CT），随访过程中如有结节增大、密度增高、实性成分增加等，需再次建议患者积极治疗（手术切除）。当然，尽管CAC大于9，CTAI＞85%，提示肺结节恶性风险高危，但手术切除后病理依然存在良性结节的可能，虽然这种可能性比较小”。
CAC > 9, CTAI 结节风险 < 85%：分析解读参考 “ CAC高风险区间，CTAI未达高风险区间，建议抗炎治疗（两周）后3个月复查胸部CT，同时进行CTAI随访对比。如有结节增大、密度增高、实性成分增加等，可建议积极治疗，如果没有结节增大、密度增高、实性成分增加等，应对患者进行密切随访，每3个月复查胸部CT，连续3-4次，随访观察过程中出现结节恶性倾向增大（结节增大、密度增高、实性成分增加或出现下列征象一项以上时：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。”
```
"""
messages_1 = []
messages_1.append({"role":"system", "content":system_prompt_1})
check_info_1 = """
患者姓名：尹玉梅；
CAC: 20;
结节信息: 
1号结节 结节风险预测：90%；结节类型：混合型；结节大小：15mm, 
2号结节 结节风险预测：80%；结节类型：磨玻璃型；结节大小：6mm；
"""
messages_1.append({"role":"user", "content":check_info_1})
import time
start_time = time.time()
current_length = 0
output = ""
async for output in vllm_gen(messages_1, top_p, temperature, max_length):
    print(output[current_length:], end="", flush=True)
    current_length = len(output)
end_time = time.time()
print('Use Time: ', end_time-start_time)

In [ ]:
output